In [1]:
import time


In [2]:
for i in range(0,5):
    print(i+1, ": Praying to code almighty for a good grade." + "I hope I don't get a meh score.")
    time.sleep(0.6)
print("\n \nHope github doesn't crash.")
print("Hope Huggingface doesn't crash.")
print("Hope my computer doesn't crash.")
print("Hope I don't crash.")
print("Hope all the above prayers work.")

1 : Praying to code almighty for a good grade.I hope I don't get a meh score.
2 : Praying to code almighty for a good grade.I hope I don't get a meh score.
3 : Praying to code almighty for a good grade.I hope I don't get a meh score.
4 : Praying to code almighty for a good grade.I hope I don't get a meh score.
5 : Praying to code almighty for a good grade.I hope I don't get a meh score.

 
Hope github doesn't crash.
Hope Huggingface doesn't crash.
Hope my computer doesn't crash.
Hope I don't crash.
Hope all the above prayers work.


In [ ]:
#install pytorch if you dont have it(cpu/cuda)
#%pip install torch torchvision
#%pip3 install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu117

In [ ]:
%pip install transformers
%pip install scipy
%pip install gradio


In [3]:
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from scipy.special import softmax
import gradio as gr

The model was trained when username was denoted as @user and the links are replaced with http. So we bow down to the arbitary model throwing tantrums and satisfy its wishes

In [4]:

# preprocess tweet as function
def preprocess(tweet):
    tweet_words = []

    for word in tweet.split(' '):
        if word.startswith('@') and len(word) > 1:
            word = '@user'
        
        elif word.startswith('http'):
            word = "http"
        tweet_words.append(word)

    tweet_proc = " ".join(tweet_words)
    return tweet_proc


In [5]:
# load model and tokenizer
#roberta = "cardiffnlp/twitter-roberta-base-sentiment" this is the base model
roberta = "sudhanvasp/Sentiment-Analysis" #this is the custom model that utilizes the base model with additional parameters to improve accuracy and performance


model = AutoModelForSequenceClassification.from_pretrained(roberta)
tokenizer = AutoTokenizer.from_pretrained(roberta)

labels = ['Negative', 'Neutral', 'Positive']


Some weights of the model checkpoint at sudhanvasp/Sentiment-Analysis were not used when initializing RobertaForSequenceClassification: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
- This IS expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [ ]:
#output as a function if you dont want gradio
def sentiment(encoded_tweet):
        tweet_proc = preprocess(tweet)
        encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')

        output = model(**encoded_tweet)

        scores = output[0][0].detach().numpy()
        scores = softmax(scores)
        #storing labels and scores in a dictionary in one line
        results = dict(zip(labels, scores))
         #print max score and label of the tweet
        print("Tweet: ", tweet)
        print("Sentiment: ", labels[scores.argmax()], scores.max())
        print("\n\n")
        print("In detail: ")
        return results
def nograd():
        for i in range(0,5):
                tweet = input("Enter a tweet: ")
                tweet_proc = preprocess(tweet)
                encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')
                sentiment(encoded_tweet)
                        

        
    

In [6]:


# Define the Gradio interface
def predict_sentiment(tweet):
    tweet_proc = preprocess(tweet)
    encoded_tweet = tokenizer(tweet_proc, return_tensors='pt')

    output = model(**encoded_tweet)

    scores = output[0][0].detach().numpy()
    scores = softmax(scores)

    # Get the sentiment label and score
    sentiment_label = labels[scores.argmax()]
    sentiment_score = scores.max()

    return sentiment_label, sentiment_score


In [8]:

# Create a Gradio interface
iface = gr.Interface(
    fn=predict_sentiment,
    inputs="text",
    outputs=[gr.components.Textbox(label="Sentiment Label"),gr.components.Textbox(label="Sentiment Score")],
    title="Sentiment Analysis",
    description="Enter a tweet, and this model will predict its sentiment.",
)

# Launch the Gradio interface
#iface.launch(share=True)
iface.launch()

Running on local URL:  http://127.0.0.1:7861

To create a public link, set `share=True` in `launch()`.
